In [21]:
import numpy as np,pandas as pd

In [206]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [207]:
def rmse_loss(y_test,y_prediction):
    result=0
    for i,k in zip(y_test,y_prediction):
        result += (i-k)**2
    return result/len(y_test)

In [208]:
def accuracy(y_test,y_prediction):
    true_count=0
    for i,k in zip(y_test,y_prediction):
         if i==round(k): true_count+=1
    return true_count/len(y_test)

In [209]:
def log_or_binary_loss(y_test,y_prediction):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_prediction]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_test*np.log(y_predicted_new)+(1-y_test)*np.log(1-y_predicted_new))

In [210]:
class myNN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold,metric):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold,metric)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold,metric):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid(weighted_sum)
            if metric == "rmse":
                loss = rmse_loss(y_true, y_predicted)
            elif metric=="log":
                loss = log_or_binary_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [211]:
df = pd.read_csv('Datasets/insurance_data.csv')

In [212]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   age               28 non-null     int64
 1   affordibility     28 non-null     int64
 2   bought_insurance  28 non-null     int64
dtypes: int64(3)
memory usage: 800.0 bytes


In [214]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.3)

In [215]:
x_test_scaled = x_test
x_test_scaled.age = x_test_scaled.age.apply(lambda x: x/100)
x_train_scaled = x_train
x_train_scaled.age = x_train_scaled.age.apply(lambda x: x/100)

## With log binary cross entropy metric

In [216]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs=7500, loss_thresold=0.5,metric="log")

Epoch:0, w1:0.9723874055827191, w2:0.9559136232351411, bias:-0.12310173771032953, loss:0.7040537211548044
Epoch:50, w1:1.3466140729252525, w2:1.4218575570218868, bias:-1.4398296885508661, loss:0.530616146868058
Epoch:100, w1:1.9019568522291712, w2:1.74174664333124, bias:-1.9157242454935308, loss:0.5044305433468942
Epoch:113, w1:2.0429662917705973, w2:1.7878200338447143, bias:-2.0096137326666117, loss:0.4996578480632344
Final weights and bias: w1: 2.0429662917705973, w2: 1.7878200338447143, bias: -2.0096137326666117


In [217]:
y_pred = customModel.predict(X_test=x_test_scaled)

In [218]:
pd.concat([round(y_pred),y_test],axis=1,keys=['my_prediction', 'true_value'])

,my_prediction,true_value
22,1.0,1
17,1.0,1
19,1.0,0
8,1.0,1
7,0.0,1
25,1.0,1
11,1.0,0
20,1.0,0
6,0.0,0


## With rmse  metric

In [225]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs=9000, loss_thresold=0.1,metric="rmse")

Epoch:0, w1:0.9723874055827191, w2:0.9559136232351411, bias:-0.12310173771032953, loss:0.2562672000589127
Epoch:50, w1:1.3466140729252525, w2:1.4218575570218868, bias:-1.4398296885508661, loss:0.17279955633147084
Epoch:100, w1:1.9019568522291712, w2:1.74174664333124, bias:-1.9157242454935308, loss:0.16177266061893772
Epoch:150, w1:2.4345407629844567, w2:1.8720277317850471, bias:-2.2357190238098434, loss:0.15458939290201346
Epoch:200, w1:2.9383981000064816, w2:1.9185630324842866, bias:-2.4767074368552566, loss:0.1487990132234758
Epoch:250, w1:3.411728527572606, w2:1.9255889182707613, bias:-2.6727762003911693, loss:0.14389141415959056
Epoch:300, w1:3.8547169951183804, w2:1.9138607527819593, bias:-2.8408741382159275, loss:0.13969100089058661
Epoch:350, w1:4.268531783705876, w2:1.893723269630109, bias:-2.9900644790098427, loss:0.1360935066703278
Epoch:400, w1:4.654809012547742, w2:1.8704535278704388, bias:-3.1254580662714426, loss:0.133014887097069
Epoch:450, w1:5.015371210940219, w2:1.846

In [226]:
y_pred = customModel.predict(X_test=x_test_scaled)

In [227]:
pd.concat([round(y_pred),y_test],axis=1,keys=['my_prediction', 'true_value'])

,my_prediction,true_value
22,1.0,1
17,1.0,1
19,0.0,0
8,1.0,1
7,1.0,1
25,1.0,1
11,0.0,0
20,0.0,0
6,1.0,0
